In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
%matplotlib inline

In [3]:
import ray

import pandas as pd 
import numpy as np
import os
import psutil

from SepMe.processing.preprocessing import preprocess_df, plot_colored_circles
import matplotlib.pyplot as plt 


print("Number of processors: ", psutil.cpu_count())
os.getcwd()

SepMe - INFO - MLFLOW_TRACKING_URI - http://localhost:5000


Number of processors:  8


'/Users/morarica/Developer/SepMe/Jupyters'

In [4]:
ray.init(num_cpus=psutil.cpu_count())

2020-01-14 15:16:53,340	INFO resource_spec.py:216 -- Starting Ray with 4.15 GiB memory available for workers and up to 2.09 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).


{'node_ip_address': '129.69.205.80',
 'redis_address': '129.69.205.80:48810',
 'object_store_address': '/tmp/ray/session_2020-01-14_15-16-53_332403_21222/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-01-14_15-16-53_332403_21222/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2020-01-14_15-16-53_332403_21222'}

In [5]:
@ray.remote
def process_1v1(path, file, i, l):

    df = pd.read_csv(path + 'data/' + file) 
    ff = path + 'figures_1v11/' + file.split('.csv')[0] + '_0.png'

#     if os.path.exists(ff):
#         print('Skipping: {} {}/{}'.format(file, i,l))
#         # print('Skipping file.... {}'.format(file))
#         return 0
        
    
    df1, circles = preprocess_df(df, ['x', 'y'], size = 0.5, 
                                            occlusion = 0.1, save=None, sort=True)
    
    
    if len(df1) == len(df):
        print(file)
    
    ddf = pd.get_dummies(df1['class'])
    
    #print(df1.shape)
    #print(ddf.shape)
    df1 = pd.concat([df1,ddf], axis=1)
    #print(df1.shape)

    for cc in ddf.columns:
        #print(cc)
        fig_name = path + 'figures_1v11/' + file.split('.csv')[0] + '_{}'.format(cc) + '.png'
        
        
        if os.path.exists(ff):
            # print('Skipping: {} {}/{}'.format(file, i,l))
            # print('Skipping file.... {}'.format(file))
            continue

        df2 = df1.copy()
        df2.drop(['class'],axis=1, inplace=True)



        #df2['class'] = 0 
        df2['class'] = df2[cc]
        #print(df2)

        fig, ax = plt.subplots(figsize = (10,10))
        plot_colored_circles(ax, df2, circles, specific_colors = int(cc))
        fig.savefig(fig_name)
        plt.close('all')
    
    print('Finished Processing: {} {}/{}'.format(file, i,l))
    return 1

In [6]:
path = '../data/mturk_samples/task/'

if not os.path.exists(path + 'figures_1v11'):
        os.makedirs(path + 'figures_1v11')
        
l = len(os.listdir(path + 'data/'))
for i, file in enumerate(os.listdir(path + 'data/')):
    if file.endswith('.csv'):
        process_1v1.remote(path, file, i, l)

(pid=21241) SepMe - INFO - MLFLOW_TRACKING_URI - http://localhost:5000
(pid=21240) SepMe - INFO - MLFLOW_TRACKING_URI - http://localhost:5000
(pid=21237) SepMe - INFO - MLFLOW_TRACKING_URI - http://localhost:5000
(pid=21242) SepMe - INFO - MLFLOW_TRACKING_URI - http://localhost:5000
(pid=21238) SepMe - INFO - MLFLOW_TRACKING_URI - http://localhost:5000
(pid=21239) SepMe - INFO - MLFLOW_TRACKING_URI - http://localhost:5000
(pid=21244) SepMe - INFO - MLFLOW_TRACKING_URI - http://localhost:5000
(pid=21243) SepMe - INFO - MLFLOW_TRACKING_URI - http://localhost:5000
(pid=21243) Finished Processing: grid6_4d_RobPCA_1-2.csv 6/126
(pid=21239) Finished Processing: iris_RobPCA_1-2.csv 4/126
(pid=21244) Finished Processing: tse300_tSNE_1-2.csv 1/126
(pid=21240) Finished Processing: ms_interleaved_60_120_3d_25_centeredClusters_tSNE_1-2.csv 2/126
(pid=21243) Finished Processing: tse300_RobPCA_1-2.csv 8/126
(pid=21243) Finished Processing: n100-d5-c3-spr0.2-out0_tSNE_1-2.csv 12/126
(pid=21242) Finis

(pid=21242) Finished Processing: ms_interleaved_60_120_3d_25_centeredClusters_GlimmerMDS_1-2.csv 87/126
(pid=21242) Finished Processing: boston_RobPCA_1-2.csv 91/126
(pid=21244) Finished Processing: JavierGeneratedData_3dinterleaved_4classes_RobPCA_1-2.csv 89/126
(pid=21244) Finished Processing: fisheries_clusteredByEscapementTarget_RobPCA_1-2.csv 92/126
(pid=21240) Finished Processing: JavierGeneratedData_3dinterleaved_3classes_GlimmerMDS_1-2.csv 90/126
(pid=21244) Finished Processing: fisheries_clusteredByEscapementTarget_GlimmerMDS_1-2.csv 94/126
(pid=21240) Finished Processing: boston_GlimmerMDS_1-2.csv 95/126
(pid=21239) Finished Processing: interleaved_100_200_10d_0_notcramped_notrotated_GlimmerMDS_1-2.csv 83/126
(pid=21239) Finished Processing: n100-d10-c3-spr0.2-out0_RobPCA_1-2.csv 98/126
(pid=21242) Finished Processing: interleaved_100_200_5d_25_cramped_rotated_tSNE_1-2.csv 93/126
(pid=21244) Finished Processing: n500-d5-c5-spr0.2-out0_tSNE_1-2.csv 96/126
(pid=21239) Finished 

In [8]:
import boto3

path = '../data/mturk_samples/task/'


files = []
idx = []

s3 = boto3.resource("s3")

for file in os.listdir(path + 'figures_1v11'):
    if file.endswith('.png'):
        files.append("https://scatterplots.s3.eu-central-1.amazonaws.com/sample_1v1/" + file)
        idx.append(int(file.split('.png')[0].split('_')[-1]))
        
        s3.meta.client.upload_file(
                    Filename=path + 'figures_1v11/' + file,
                    Bucket='scatterplots',
                    Key="sample_1v1/" + file
            ,
                )
        
        
df = pd.DataFrame({'image_url':files, 'i':idx})
df.loc[df.i<4,:].to_csv(path + 'index_1v1.csv', index=False)

In [14]:
len(df.loc[df.i<4,:])*2*20*0.02

366.40000000000003

In [9]:
df.loc[df.i<4,:]

,image_url,i
1,https://scatterplots.s3.eu-central-1.amazonaws...,0
2,https://scatterplots.s3.eu-central-1.amazonaws...,3
3,https://scatterplots.s3.eu-central-1.amazonaws...,2
4,https://scatterplots.s3.eu-central-1.amazonaws...,3
5,https://scatterplots.s3.eu-central-1.amazonaws...,1
...,...,...
701,https://scatterplots.s3.eu-central-1.amazonaws...,2
702,https://scatterplots.s3.eu-central-1.amazonaws...,0
703,https://scatterplots.s3.eu-central-1.amazonaws...,3
704,https://scatterplots.s3.eu-central-1.amazonaws...,1
